In [1]:

import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

In [2]:
dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556])


In [3]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

140
500
1000


# GCN using only domain Feature

In [26]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [3]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [ ]:
pred=torch.tensor([1, 0, 1,0])
label=torch.tensor([1, 1, 1,0])
correct = pred.eq(label).sum().item()
total=len(label)
print(correct)
print(total)
print(correct/total)
print(ACC(pred,label))

In [4]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Processing...
Done!


Run: 01, Epoch: 01, Loss: 2.3461, Train: 48.57%, Valid: 35.60% Test: 38.70%
Run: 01, Epoch: 02, Loss: 1.5001, Train: 77.86%, Valid: 56.60% Test: 60.30%
Run: 01, Epoch: 03, Loss: 1.1152, Train: 90.00%, Valid: 64.00% Test: 66.90%
Run: 01, Epoch: 04, Loss: 0.9433, Train: 94.29%, Valid: 66.80% Test: 70.10%
Run: 01, Epoch: 05, Loss: 0.8134, Train: 97.86%, Valid: 69.20% Test: 71.60%
Run: 01, Epoch: 06, Loss: 0.7259, Train: 97.86%, Valid: 71.80% Test: 72.40%
Run: 01, Epoch: 07, Loss: 0.6297, Train: 97.86%, Valid: 71.40% Test: 73.30%
Run: 01, Epoch: 08, Loss: 0.5426, Train: 98.57%, Valid: 71.20% Test: 72.70%
Run: 01, Epoch: 09, Loss: 0.5027, Train: 98.57%, Valid: 70.60% Test: 73.20%
Run: 01, Epoch: 10, Loss: 0.4358, Train: 98.57%, Valid: 71.00% Test: 72.80%
Run: 01, Epoch: 11, Loss: 0.3901, Train: 98.57%, Valid: 71.00% Test: 72.90%
Run: 01, Epoch: 12, Loss: 0.3628, Train: 98.57%, Valid: 70.80% Test: 72.50%
Run: 01, Epoch: 13, Loss: 0.3141, Train: 100.00%, Valid: 70.80% Test: 72.60%
Run: 01, Ep

Run: 01, Epoch: 133, Loss: 0.0050, Train: 100.00%, Valid: 65.20% Test: 68.00%
Run: 01, Epoch: 134, Loss: 0.0284, Train: 100.00%, Valid: 65.20% Test: 67.90%
Run: 01, Epoch: 135, Loss: 0.0090, Train: 100.00%, Valid: 65.20% Test: 67.90%
Run: 01, Epoch: 136, Loss: 0.0099, Train: 100.00%, Valid: 65.00% Test: 67.90%
Run: 01, Epoch: 137, Loss: 0.0203, Train: 100.00%, Valid: 65.00% Test: 68.00%
Run: 01, Epoch: 138, Loss: 0.0248, Train: 100.00%, Valid: 65.00% Test: 68.00%
Run: 01, Epoch: 139, Loss: 0.0068, Train: 100.00%, Valid: 65.00% Test: 68.00%
Run: 01, Epoch: 140, Loss: 0.0038, Train: 100.00%, Valid: 65.00% Test: 68.40%
Run: 01, Epoch: 141, Loss: 0.0106, Train: 100.00%, Valid: 64.80% Test: 68.30%
Run: 01, Epoch: 142, Loss: 0.0100, Train: 100.00%, Valid: 64.80% Test: 68.20%
Run: 01, Epoch: 143, Loss: 0.0060, Train: 100.00%, Valid: 64.40% Test: 68.20%
Run: 01, Epoch: 144, Loss: 0.0132, Train: 100.00%, Valid: 64.40% Test: 68.20%
Run: 01, Epoch: 145, Loss: 0.0085, Train: 100.00%, Valid: 64.60%

Run: 02, Epoch: 67, Loss: 0.0161, Train: 100.00%, Valid: 68.60% Test: 70.90%
Run: 02, Epoch: 68, Loss: 0.0291, Train: 100.00%, Valid: 68.60% Test: 70.90%
Run: 02, Epoch: 69, Loss: 0.0336, Train: 100.00%, Valid: 68.60% Test: 70.90%
Run: 02, Epoch: 70, Loss: 0.0221, Train: 100.00%, Valid: 68.60% Test: 70.70%
Run: 02, Epoch: 71, Loss: 0.0130, Train: 100.00%, Valid: 68.40% Test: 70.50%
Run: 02, Epoch: 72, Loss: 0.0200, Train: 100.00%, Valid: 68.20% Test: 70.20%
Run: 02, Epoch: 73, Loss: 0.0295, Train: 100.00%, Valid: 68.20% Test: 70.20%
Run: 02, Epoch: 74, Loss: 0.0369, Train: 100.00%, Valid: 68.40% Test: 70.30%
Run: 02, Epoch: 75, Loss: 0.0132, Train: 100.00%, Valid: 68.40% Test: 70.30%
Run: 02, Epoch: 76, Loss: 0.0082, Train: 100.00%, Valid: 68.40% Test: 70.10%
Run: 02, Epoch: 77, Loss: 0.0198, Train: 100.00%, Valid: 68.60% Test: 70.30%
Run: 02, Epoch: 78, Loss: 0.0288, Train: 100.00%, Valid: 68.40% Test: 70.40%
Run: 02, Epoch: 79, Loss: 0.0104, Train: 100.00%, Valid: 68.20% Test: 70.40%

Run: 02, Epoch: 180, Loss: 0.0380, Train: 100.00%, Valid: 64.80% Test: 66.70%
Run: 02, Epoch: 181, Loss: 0.0146, Train: 100.00%, Valid: 64.80% Test: 66.80%
Run: 02, Epoch: 182, Loss: 0.0030, Train: 100.00%, Valid: 64.80% Test: 66.80%
Run: 02, Epoch: 183, Loss: 0.0097, Train: 100.00%, Valid: 64.80% Test: 66.70%
Run: 02, Epoch: 184, Loss: 0.0190, Train: 100.00%, Valid: 64.80% Test: 66.60%
Run: 02, Epoch: 185, Loss: 0.0186, Train: 100.00%, Valid: 64.80% Test: 66.30%
Run: 02, Epoch: 186, Loss: 0.0035, Train: 100.00%, Valid: 64.60% Test: 66.20%
Run: 02, Epoch: 187, Loss: 0.0219, Train: 100.00%, Valid: 64.80% Test: 66.20%
Run: 02, Epoch: 188, Loss: 0.0109, Train: 100.00%, Valid: 64.80% Test: 66.20%
Run: 02, Epoch: 189, Loss: 0.0061, Train: 100.00%, Valid: 64.80% Test: 66.10%
Run: 02, Epoch: 190, Loss: 0.0039, Train: 100.00%, Valid: 64.60% Test: 66.00%
Run: 02, Epoch: 191, Loss: 0.0075, Train: 100.00%, Valid: 64.60% Test: 65.90%
Run: 02, Epoch: 192, Loss: 0.0056, Train: 100.00%, Valid: 64.60%

...

Run: 08, Epoch: 143, Loss: 0.0109, Train: 100.00%, Valid: 63.60% Test: 65.30%
Run: 08, Epoch: 144, Loss: 0.0143, Train: 100.00%, Valid: 63.40% Test: 65.40%
Run: 08, Epoch: 145, Loss: 0.0134, Train: 100.00%, Valid: 63.40% Test: 65.30%
Run: 08, Epoch: 146, Loss: 0.0079, Train: 100.00%, Valid: 63.40% Test: 65.10%
Run: 08, Epoch: 147, Loss: 0.0068, Train: 100.00%, Valid: 63.20% Test: 65.10%
Run: 08, Epoch: 148, Loss: 0.0049, Train: 100.00%, Valid: 62.80% Test: 65.10%
Run: 08, Epoch: 149, Loss: 0.0014, Train: 100.00%, Valid: 62.80% Test: 65.00%
Run: 08, Epoch: 150, Loss: 0.0028, Train: 100.00%, Valid: 62.80% Test: 65.20%
Run: 08, Epoch: 151, Loss: 0.0067, Train: 100.00%, Valid: 62.60% Test: 65.10%
Run: 08, Epoch: 152, Loss: 0.0054, Train: 100.00%, Valid: 62.60% Test: 65.00%
Run: 08, Epoch: 153, Loss: 0.0113, Train: 100.00%, Valid: 62.60% Test: 65.10%
Run: 08, Epoch: 154, Loss: 0.0044, Train: 100.00%, Valid: 63.00% Test: 65.00%
Run: 08, Epoch: 155, Loss: 0.0175, Train: 100.00%, Valid: 63.00%

Run: 09, Epoch: 50, Loss: 0.0156, Train: 100.00%, Valid: 69.60% Test: 71.90%
Run: 09, Epoch: 51, Loss: 0.0121, Train: 100.00%, Valid: 69.60% Test: 72.10%
Run: 09, Epoch: 52, Loss: 0.0142, Train: 100.00%, Valid: 69.80% Test: 72.00%
Run: 09, Epoch: 53, Loss: 0.0230, Train: 100.00%, Valid: 69.80% Test: 72.10%
Run: 09, Epoch: 54, Loss: 0.0320, Train: 100.00%, Valid: 69.80% Test: 71.90%
Run: 09, Epoch: 55, Loss: 0.0162, Train: 100.00%, Valid: 69.80% Test: 71.70%
Run: 09, Epoch: 56, Loss: 0.0273, Train: 100.00%, Valid: 69.60% Test: 71.80%
Run: 09, Epoch: 57, Loss: 0.0269, Train: 100.00%, Valid: 69.80% Test: 71.80%
Run: 09, Epoch: 58, Loss: 0.0345, Train: 100.00%, Valid: 69.40% Test: 71.80%
Run: 09, Epoch: 59, Loss: 0.0182, Train: 100.00%, Valid: 69.20% Test: 71.70%
Run: 09, Epoch: 60, Loss: 0.0262, Train: 100.00%, Valid: 69.20% Test: 71.70%
Run: 09, Epoch: 61, Loss: 0.0245, Train: 100.00%, Valid: 69.20% Test: 71.70%
Run: 09, Epoch: 62, Loss: 0.0344, Train: 100.00%, Valid: 69.20% Test: 71.90%

Run: 09, Epoch: 170, Loss: 0.0130, Train: 100.00%, Valid: 67.60% Test: 68.50%
Run: 09, Epoch: 171, Loss: 0.0046, Train: 100.00%, Valid: 68.00% Test: 68.60%
Run: 09, Epoch: 172, Loss: 0.0074, Train: 100.00%, Valid: 68.20% Test: 68.50%
Run: 09, Epoch: 173, Loss: 0.0018, Train: 100.00%, Valid: 68.60% Test: 68.50%
Run: 09, Epoch: 174, Loss: 0.0055, Train: 100.00%, Valid: 68.80% Test: 68.70%
Run: 09, Epoch: 175, Loss: 0.0078, Train: 100.00%, Valid: 68.80% Test: 68.50%
Run: 09, Epoch: 176, Loss: 0.0071, Train: 100.00%, Valid: 68.80% Test: 68.80%
Run: 09, Epoch: 177, Loss: 0.0222, Train: 100.00%, Valid: 67.80% Test: 69.00%
Run: 09, Epoch: 178, Loss: 0.0047, Train: 100.00%, Valid: 67.80% Test: 68.80%
Run: 09, Epoch: 179, Loss: 0.0120, Train: 100.00%, Valid: 67.80% Test: 68.70%
Run: 09, Epoch: 180, Loss: 0.0184, Train: 100.00%, Valid: 67.60% Test: 68.60%
Run: 09, Epoch: 181, Loss: 0.0051, Train: 100.00%, Valid: 67.00% Test: 68.30%
Run: 09, Epoch: 182, Loss: 0.0383, Train: 100.00%, Valid: 66.60%

Run: 10, Epoch: 77, Loss: 0.0225, Train: 100.00%, Valid: 69.40% Test: 70.30%
Run: 10, Epoch: 78, Loss: 0.0312, Train: 100.00%, Valid: 69.60% Test: 70.40%
Run: 10, Epoch: 79, Loss: 0.0163, Train: 100.00%, Valid: 69.40% Test: 70.30%
Run: 10, Epoch: 80, Loss: 0.0059, Train: 100.00%, Valid: 69.20% Test: 70.20%
Run: 10, Epoch: 81, Loss: 0.0062, Train: 100.00%, Valid: 69.20% Test: 70.20%
Run: 10, Epoch: 82, Loss: 0.0193, Train: 100.00%, Valid: 69.20% Test: 70.40%
Run: 10, Epoch: 83, Loss: 0.0124, Train: 100.00%, Valid: 69.20% Test: 70.30%
Run: 10, Epoch: 84, Loss: 0.0079, Train: 100.00%, Valid: 69.20% Test: 70.20%
Run: 10, Epoch: 85, Loss: 0.0339, Train: 100.00%, Valid: 69.20% Test: 70.30%
Run: 10, Epoch: 86, Loss: 0.0111, Train: 100.00%, Valid: 69.20% Test: 70.30%
Run: 10, Epoch: 87, Loss: 0.0571, Train: 100.00%, Valid: 69.40% Test: 70.30%
Run: 10, Epoch: 88, Loss: 0.0120, Train: 100.00%, Valid: 69.40% Test: 70.30%
Run: 10, Epoch: 89, Loss: 0.0279, Train: 100.00%, Valid: 69.20% Test: 70.40%

Run: 10, Epoch: 187, Loss: 0.0122, Train: 100.00%, Valid: 66.40% Test: 68.10%
Run: 10, Epoch: 188, Loss: 0.0029, Train: 100.00%, Valid: 66.40% Test: 68.10%
Run: 10, Epoch: 189, Loss: 0.0071, Train: 100.00%, Valid: 66.20% Test: 68.10%
Run: 10, Epoch: 190, Loss: 0.0060, Train: 100.00%, Valid: 66.20% Test: 68.10%
Run: 10, Epoch: 191, Loss: 0.0030, Train: 100.00%, Valid: 66.20% Test: 68.20%
Run: 10, Epoch: 192, Loss: 0.0088, Train: 100.00%, Valid: 66.20% Test: 68.10%
Run: 10, Epoch: 193, Loss: 0.0044, Train: 100.00%, Valid: 66.20% Test: 68.10%
Run: 10, Epoch: 194, Loss: 0.0052, Train: 100.00%, Valid: 66.20% Test: 68.30%
Run: 10, Epoch: 195, Loss: 0.0087, Train: 100.00%, Valid: 66.40% Test: 68.20%
Run: 10, Epoch: 196, Loss: 0.0205, Train: 100.00%, Valid: 66.60% Test: 68.20%
Run: 10, Epoch: 197, Loss: 0.0034, Train: 100.00%, Valid: 66.60% Test: 68.30%
Run: 10, Epoch: 198, Loss: 0.0173, Train: 100.00%, Valid: 66.60% Test: 68.20%
Run: 10, Epoch: 199, Loss: 0.0021, Train: 100.00%, Valid: 66.80%

# WISE EMBEDDING

In [4]:
dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556])


In [5]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [6]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [9]:
def Similarity(array1, array2):
    intersection = np.sum(np.logical_and(array1, array2))
    return intersection
Fec=[]
for i in range(Number_nodes):
#for i in range(2):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    vec.append(Similarity(f,basis[0]))
    vec.append(Similarity(f,basis[1]))
    vec.append(Similarity(f,basis[2]))
    vec.append(Similarity(f,basis[3]))
    vec.append(Similarity(f,basis[4]))
    vec.append(Similarity(f,basis[5]))
    vec.append(Similarity(f,basis[6]))
    #print(f)
    f.clear()
    Fec.append(vec)
SFec=[]
for i in range(Number_nodes):
#for i in range(2):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    Svec.append(Similarity(f,sel_basis[0]))
    Svec.append(Similarity(f,sel_basis[1]))
    Svec.append(Similarity(f,sel_basis[2]))
    Svec.append(Similarity(f,sel_basis[3]))
    Svec.append(Similarity(f,sel_basis[4]))
    Svec.append(Similarity(f,sel_basis[5]))
    Svec.append(Similarity(f,sel_basis[6]))
    #print(f)
    f.clear()
    SFec.append(Svec)

In [11]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[ 9.,  7.,  9.,  ...,  1.,  0.,  0.],
        [21., 20., 21.,  ...,  4.,  4.,  3.],
        [19., 17., 18.,  ...,  6.,  4.,  4.],
        ...,
        [16., 16., 15.,  ...,  3.,  5.,  3.],
        [12.,  9., 11.,  ...,  2.,  2.,  1.],
        [10., 12.,  8.,  ...,  2.,  1.,  1.]])


'torch.FloatTensor'

# W-GCN 

In [14]:
data.x=CC_domain
print(data)

Data(x=[2708, 14], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556])


In [15]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.3444, Train: 14.29%, Valid: 31.60% Test: 31.90%
Run: 01, Epoch: 02, Loss: 2.2768, Train: 14.29%, Valid: 31.40% Test: 31.90%
Run: 01, Epoch: 03, Loss: 2.1548, Train: 14.29%, Valid: 31.40% Test: 31.80%
Run: 01, Epoch: 04, Loss: 2.0771, Train: 13.57%, Valid: 31.00% Test: 31.50%
Run: 01, Epoch: 05, Loss: 1.9681, Train: 13.57%, Valid: 30.00% Test: 30.50%
Run: 01, Epoch: 06, Loss: 1.9588, Train: 17.14%, Valid: 29.00% Test: 29.30%
Run: 01, Epoch: 07, Loss: 1.9285, Train: 17.86%, Valid: 17.40% Test: 20.50%
Run: 01, Epoch: 08, Loss: 1.8786, Train: 16.43%, Valid: 9.80% Test: 11.80%
Run: 01, Epoch: 09, Loss: 1.8286, Train: 17.14%, Valid: 10.00% Test: 11.40%
Run: 01, Epoch: 10, Loss: 1.7913, Train: 21.43%, Valid: 13.20% Test: 15.20%
Run: 01, Epoch: 11, Loss: 1.7364, Train: 21.43%, Valid: 17.00% Test: 17.70%
Run: 01, Epoch: 12, Loss: 1.7552, Train: 23.57%, Valid: 17.60% Test: 17.70%
Run: 01, Epoch: 13, Loss: 1.7313, Train: 20.71%, Valid: 18.20% Test: 17.60%
Run: 01, Epoc

Run: 01, Epoch: 160, Loss: 0.2718, Train: 88.57%, Valid: 82.60% Test: 85.00%
Run: 01, Epoch: 161, Loss: 0.2338, Train: 89.29%, Valid: 81.60% Test: 84.70%
Run: 01, Epoch: 162, Loss: 0.2966, Train: 92.14%, Valid: 82.20% Test: 84.20%
Run: 01, Epoch: 163, Loss: 0.2386, Train: 90.71%, Valid: 79.40% Test: 83.60%
Run: 01, Epoch: 164, Loss: 0.2252, Train: 88.57%, Valid: 78.80% Test: 82.50%
Run: 01, Epoch: 165, Loss: 0.2479, Train: 92.86%, Valid: 79.00% Test: 83.50%
Run: 01, Epoch: 166, Loss: 0.2723, Train: 93.57%, Valid: 79.80% Test: 85.40%
Run: 01, Epoch: 167, Loss: 0.2133, Train: 90.00%, Valid: 80.40% Test: 82.40%
Run: 01, Epoch: 168, Loss: 0.2486, Train: 90.71%, Valid: 79.80% Test: 82.50%
Run: 01, Epoch: 169, Loss: 0.2611, Train: 92.14%, Valid: 80.00% Test: 83.90%
Run: 01, Epoch: 170, Loss: 0.2446, Train: 95.00%, Valid: 82.40% Test: 82.50%
Run: 01, Epoch: 171, Loss: 0.2647, Train: 92.86%, Valid: 77.80% Test: 81.10%
Run: 01, Epoch: 172, Loss: 0.2151, Train: 94.29%, Valid: 78.00% Test: 81.20%

Run: 02, Epoch: 135, Loss: 0.2533, Train: 95.00%, Valid: 82.20% Test: 85.20%
Run: 02, Epoch: 136, Loss: 0.2673, Train: 95.00%, Valid: 83.00% Test: 84.80%
Run: 02, Epoch: 137, Loss: 0.3236, Train: 94.29%, Valid: 82.40% Test: 85.90%
Run: 02, Epoch: 138, Loss: 0.3303, Train: 93.57%, Valid: 82.40% Test: 83.90%
Run: 02, Epoch: 139, Loss: 0.2704, Train: 87.86%, Valid: 76.80% Test: 77.10%
Run: 02, Epoch: 140, Loss: 0.3042, Train: 82.86%, Valid: 71.00% Test: 71.90%
Run: 02, Epoch: 141, Loss: 0.2729, Train: 80.00%, Valid: 71.20% Test: 72.30%
Run: 02, Epoch: 142, Loss: 0.3378, Train: 87.14%, Valid: 77.40% Test: 77.70%
Run: 02, Epoch: 143, Loss: 0.3286, Train: 96.43%, Valid: 83.80% Test: 85.10%
Run: 02, Epoch: 144, Loss: 0.2793, Train: 93.57%, Valid: 83.40% Test: 85.60%
Run: 02, Epoch: 145, Loss: 0.2950, Train: 90.71%, Valid: 82.20% Test: 83.70%
Run: 02, Epoch: 146, Loss: 0.2197, Train: 89.29%, Valid: 82.00% Test: 83.50%
Run: 02, Epoch: 147, Loss: 0.2606, Train: 86.43%, Valid: 80.80% Test: 82.10%

Run: 03, Epoch: 112, Loss: 0.3136, Train: 91.43%, Valid: 79.80% Test: 83.50%
Run: 03, Epoch: 113, Loss: 0.3011, Train: 87.14%, Valid: 79.60% Test: 81.50%
Run: 03, Epoch: 114, Loss: 0.3364, Train: 87.14%, Valid: 79.60% Test: 81.50%
Run: 03, Epoch: 115, Loss: 0.2860, Train: 90.00%, Valid: 80.00% Test: 82.90%
Run: 03, Epoch: 116, Loss: 0.2738, Train: 89.29%, Valid: 82.00% Test: 83.00%
Run: 03, Epoch: 117, Loss: 0.3027, Train: 88.57%, Valid: 80.60% Test: 81.20%
Run: 03, Epoch: 118, Loss: 0.3027, Train: 91.43%, Valid: 80.60% Test: 80.30%
Run: 03, Epoch: 119, Loss: 0.2702, Train: 90.71%, Valid: 81.00% Test: 79.80%
Run: 03, Epoch: 120, Loss: 0.2964, Train: 90.71%, Valid: 79.40% Test: 79.00%
Run: 03, Epoch: 121, Loss: 0.2664, Train: 91.43%, Valid: 82.00% Test: 81.80%
Run: 03, Epoch: 122, Loss: 0.2916, Train: 91.43%, Valid: 82.40% Test: 83.10%
Run: 03, Epoch: 123, Loss: 0.3141, Train: 89.29%, Valid: 82.00% Test: 82.00%
Run: 03, Epoch: 124, Loss: 0.3065, Train: 91.43%, Valid: 81.40% Test: 84.50%

Run: 04, Epoch: 82, Loss: 0.4972, Train: 84.29%, Valid: 78.80% Test: 82.90%
Run: 04, Epoch: 83, Loss: 0.4075, Train: 87.86%, Valid: 80.80% Test: 83.20%
Run: 04, Epoch: 84, Loss: 0.4914, Train: 90.00%, Valid: 84.20% Test: 84.70%
Run: 04, Epoch: 85, Loss: 0.3942, Train: 90.71%, Valid: 84.60% Test: 85.90%
Run: 04, Epoch: 86, Loss: 0.4235, Train: 87.86%, Valid: 84.00% Test: 83.70%
Run: 04, Epoch: 87, Loss: 0.3967, Train: 81.43%, Valid: 76.60% Test: 75.00%
Run: 04, Epoch: 88, Loss: 0.4154, Train: 82.86%, Valid: 76.00% Test: 74.30%
Run: 04, Epoch: 89, Loss: 0.4441, Train: 91.43%, Valid: 79.20% Test: 79.40%
Run: 04, Epoch: 90, Loss: 0.4236, Train: 93.57%, Valid: 80.20% Test: 82.90%
Run: 04, Epoch: 91, Loss: 0.4470, Train: 90.71%, Valid: 80.40% Test: 82.10%
Run: 04, Epoch: 92, Loss: 0.3631, Train: 89.29%, Valid: 79.80% Test: 81.70%
Run: 04, Epoch: 93, Loss: 0.4304, Train: 90.71%, Valid: 79.80% Test: 82.80%
Run: 04, Epoch: 94, Loss: 0.4335, Train: 94.29%, Valid: 82.40% Test: 84.90%
Run: 04, Epo

...

Run: 07, Epoch: 05, Loss: 1.8707, Train: 16.43%, Valid: 12.20% Test: 11.70%
Run: 07, Epoch: 06, Loss: 1.8324, Train: 16.43%, Valid: 12.00% Test: 12.00%
Run: 07, Epoch: 07, Loss: 1.7924, Train: 17.86%, Valid: 13.60% Test: 12.70%
Run: 07, Epoch: 08, Loss: 1.7172, Train: 20.71%, Valid: 16.40% Test: 15.80%
Run: 07, Epoch: 09, Loss: 1.7650, Train: 24.29%, Valid: 18.60% Test: 18.90%
Run: 07, Epoch: 10, Loss: 1.6659, Train: 26.43%, Valid: 20.40% Test: 21.10%
Run: 07, Epoch: 11, Loss: 1.6885, Train: 30.71%, Valid: 22.20% Test: 22.60%
Run: 07, Epoch: 12, Loss: 1.6419, Train: 29.29%, Valid: 21.20% Test: 22.70%
Run: 07, Epoch: 13, Loss: 1.5411, Train: 24.29%, Valid: 17.80% Test: 18.80%
Run: 07, Epoch: 14, Loss: 1.5617, Train: 22.14%, Valid: 18.60% Test: 18.20%
Run: 07, Epoch: 15, Loss: 1.5230, Train: 29.29%, Valid: 29.00% Test: 29.40%
Run: 07, Epoch: 16, Loss: 1.3793, Train: 38.57%, Valid: 34.20% Test: 33.10%
Run: 07, Epoch: 17, Loss: 1.4323, Train: 30.71%, Valid: 29.80% Test: 29.70%
Run: 07, Epo

Run: 07, Epoch: 178, Loss: 0.3049, Train: 77.86%, Valid: 71.20% Test: 71.70%
Run: 07, Epoch: 179, Loss: 0.2883, Train: 83.57%, Valid: 74.80% Test: 76.80%
Run: 07, Epoch: 180, Loss: 0.2624, Train: 91.43%, Valid: 79.20% Test: 81.50%
Run: 07, Epoch: 181, Loss: 0.2930, Train: 92.86%, Valid: 82.20% Test: 84.10%
Run: 07, Epoch: 182, Loss: 0.2315, Train: 90.71%, Valid: 81.40% Test: 83.00%
Run: 07, Epoch: 183, Loss: 0.2218, Train: 89.29%, Valid: 79.20% Test: 80.20%
Run: 07, Epoch: 184, Loss: 0.2306, Train: 89.29%, Valid: 79.80% Test: 80.20%
Run: 07, Epoch: 185, Loss: 0.2466, Train: 93.57%, Valid: 83.60% Test: 82.90%
Run: 07, Epoch: 186, Loss: 0.2292, Train: 87.14%, Valid: 78.20% Test: 78.20%
Run: 07, Epoch: 187, Loss: 0.1903, Train: 86.43%, Valid: 76.60% Test: 76.70%
Run: 07, Epoch: 188, Loss: 0.2488, Train: 92.86%, Valid: 82.20% Test: 82.10%
Run: 07, Epoch: 189, Loss: 0.3013, Train: 94.29%, Valid: 82.80% Test: 83.50%
Run: 07, Epoch: 190, Loss: 0.1960, Train: 91.43%, Valid: 79.80% Test: 79.60%

Run: 08, Epoch: 154, Loss: 0.3694, Train: 90.00%, Valid: 82.80% Test: 84.10%
Run: 08, Epoch: 155, Loss: 0.3541, Train: 85.71%, Valid: 78.40% Test: 78.60%
Run: 08, Epoch: 156, Loss: 0.3155, Train: 83.57%, Valid: 76.60% Test: 76.30%
Run: 08, Epoch: 157, Loss: 0.3511, Train: 82.86%, Valid: 76.60% Test: 76.70%
Run: 08, Epoch: 158, Loss: 0.3057, Train: 85.71%, Valid: 80.40% Test: 79.80%
Run: 08, Epoch: 159, Loss: 0.2861, Train: 87.14%, Valid: 80.20% Test: 82.50%
Run: 08, Epoch: 160, Loss: 0.3184, Train: 89.29%, Valid: 80.00% Test: 82.20%
Run: 08, Epoch: 161, Loss: 0.3200, Train: 90.00%, Valid: 79.20% Test: 79.30%
Run: 08, Epoch: 162, Loss: 0.3654, Train: 89.29%, Valid: 78.40% Test: 78.70%
Run: 08, Epoch: 163, Loss: 0.2809, Train: 91.43%, Valid: 79.20% Test: 80.60%
Run: 08, Epoch: 164, Loss: 0.3465, Train: 94.29%, Valid: 81.60% Test: 83.10%
Run: 08, Epoch: 165, Loss: 0.3217, Train: 92.14%, Valid: 82.00% Test: 83.20%
Run: 08, Epoch: 166, Loss: 0.2905, Train: 90.00%, Valid: 81.40% Test: 80.30%

Run: 09, Epoch: 131, Loss: 0.2751, Train: 88.57%, Valid: 75.00% Test: 75.20%
Run: 09, Epoch: 132, Loss: 0.3022, Train: 87.14%, Valid: 74.20% Test: 75.20%
Run: 09, Epoch: 133, Loss: 0.3530, Train: 92.14%, Valid: 78.00% Test: 78.80%
Run: 09, Epoch: 134, Loss: 0.2791, Train: 87.86%, Valid: 80.00% Test: 81.10%
Run: 09, Epoch: 135, Loss: 0.2664, Train: 85.00%, Valid: 81.40% Test: 79.80%
Run: 09, Epoch: 136, Loss: 0.2388, Train: 82.14%, Valid: 79.60% Test: 78.00%
Run: 09, Epoch: 137, Loss: 0.2622, Train: 81.43%, Valid: 79.80% Test: 78.90%
Run: 09, Epoch: 138, Loss: 0.2680, Train: 87.14%, Valid: 81.60% Test: 81.40%
Run: 09, Epoch: 139, Loss: 0.2843, Train: 92.14%, Valid: 82.20% Test: 83.70%
Run: 09, Epoch: 140, Loss: 0.2893, Train: 90.71%, Valid: 82.40% Test: 83.70%
Run: 09, Epoch: 141, Loss: 0.2098, Train: 90.00%, Valid: 79.80% Test: 83.10%
Run: 09, Epoch: 142, Loss: 0.3034, Train: 86.43%, Valid: 79.00% Test: 80.60%
Run: 09, Epoch: 143, Loss: 0.3440, Train: 85.71%, Valid: 77.20% Test: 78.60%

Run: 10, Epoch: 104, Loss: 0.2801, Train: 89.29%, Valid: 80.00% Test: 84.50%
Run: 10, Epoch: 105, Loss: 0.2695, Train: 90.00%, Valid: 80.80% Test: 84.80%
Run: 10, Epoch: 106, Loss: 0.3234, Train: 88.57%, Valid: 79.80% Test: 84.50%
Run: 10, Epoch: 107, Loss: 0.3400, Train: 87.86%, Valid: 79.40% Test: 81.10%
Run: 10, Epoch: 108, Loss: 0.2895, Train: 86.43%, Valid: 79.40% Test: 80.00%
Run: 10, Epoch: 109, Loss: 0.3666, Train: 87.86%, Valid: 80.20% Test: 79.60%
Run: 10, Epoch: 110, Loss: 0.3307, Train: 86.43%, Valid: 81.20% Test: 81.60%
Run: 10, Epoch: 111, Loss: 0.3371, Train: 88.57%, Valid: 82.20% Test: 83.50%
Run: 10, Epoch: 112, Loss: 0.2968, Train: 90.71%, Valid: 82.00% Test: 84.10%
Run: 10, Epoch: 113, Loss: 0.2440, Train: 90.71%, Valid: 80.20% Test: 81.70%
Run: 10, Epoch: 114, Loss: 0.3226, Train: 84.29%, Valid: 78.20% Test: 79.70%
Run: 10, Epoch: 115, Loss: 0.2439, Train: 82.86%, Valid: 78.80% Test: 80.10%
Run: 10, Epoch: 116, Loss: 0.3709, Train: 92.14%, Valid: 78.20% Test: 82.00%

# Topological Encodding 

In [12]:
dataset = Planetoid(root='/tmp/cora', name='Cora')
data = dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [19]:
print(data.edge_index.numpy())

[[ 633 1862 2582 ...  598 1473 2706]
 [   0    0    0 ... 2707 2707 2707]]


In [13]:
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [14]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [20]:
print(G.number_of_edges())

10556


In [15]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [16]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [17]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

2  |  485 

4  |  583 

6  |  553 

8  |  389 

10  |  281 

12  |  131 

14  |  82 

16  |  57 

18  |  25 

20  |  26 

22  |  14 

24  |  18 

26  |  5 

28  |  6 

30  |  6 

32  |  7 

34  |  8 

36  |  3 

38  |  5 

42  |  3 

44  |  1 

46  |  3 

52  |  1 

58  |  1 

60  |  2 

62  |  1 

64  |  2 

66  |  1 

68  |  1 

72  |  1 

80  |  1 

84  |  1 

88  |  1 

130  |  1 

148  |  1 

156  |  1 

336  |  1 



In [18]:
import pyflagser
Node_fil=[2,4,6,8,10,12,14,16,18,20,22,24,30,34]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=True, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 2707 (100%)

In [ ]:
print( Node_Edge)

In [32]:
dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556])


In [20]:
topo_betti0=torch.tensor(topo_betti_0).float()
topo_betti1=torch.tensor(topo_betti_1).float()
NodeEdge=torch.tensor(Node_Edge).float()

# TOPO-W_GCN

In [33]:
data.x=CC_domain
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556], topo=[2708, 28])


In [176]:
print(len(topo_fe[0]))

28


In [29]:

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        #return x.log_softmax(dim=-1)
        return x

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d


In [37]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.2,'num_layers_mlp': 3}

    args = objectview(args)
    #print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                #print(f'Run: {run + 1:02d}, 'f'Epoch: {epoch:02d}, 'f'Loss: {loss:.4f}, 'f'Train: {100 * train_acc:.2f}%, '
                #      f'Valid: {100 * valid_acc:.2f}% '
                 #     f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run 01:
Highest Train: 100.00
Highest Valid: 76.20
  Final Train: 100.00
   Final Test: 76.90
Run 02:
Highest Train: 100.00
Highest Valid: 73.00
  Final Train: 100.00
   Final Test: 74.10
Run 03:
Highest Train: 100.00
Highest Valid: 70.60
  Final Train: 100.00
   Final Test: 72.30
Run 04:
Highest Train: 100.00
Highest Valid: 79.20
  Final Train: 100.00
   Final Test: 78.60
Run 05:
Highest Train: 100.00
Highest Valid: 75.00
  Final Train: 100.00
   Final Test: 76.10
Run 06:
Highest Train: 100.00
Highest Valid: 73.80
  Final Train: 100.00
   Final Test: 76.20
Run 07:
Highest Train: 100.00
Highest Valid: 75.20
  Final Train: 100.00
   Final Test: 75.20
Run 08:
Highest Train: 100.00
Highest Valid: 73.20
  Final Train: 100.00
   Final Test: 74.80
Run 09:
Highest Train: 100.00
Highest Valid: 73.20
  Final Train: 100.00
   Final Test: 74.00
Run 10:
Highest Train: 100.00
Highest Valid: 73.80
  Final Train: 100.00
   Final Test: 74.90
All runs:
Highest Train: 100.00 ± 0.00
Highest Valid: 74.32 

# TOPO-GCN

In [ ]:
dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
data = dataset[0]
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

In [ ]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 1, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.2,'num_layers_mlp': 3}

    args = objectview(args)
    #print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GCN(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                #print(f'Run: {run + 1:02d}, 'f'Epoch: {epoch:02d}, 'f'Loss: {loss:.4f}, 'f'Train: {100 * train_acc:.2f}%, '
                #      f'Valid: {100 * valid_acc:.2f}% '
                 #     f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()